# Observational Study Using DoWhy - Lalonde Case Study

## Intro

After the DoWhy Package's brief presentation on the previous notebook, this one's purpose will be to demonstrate its possibilities on a real-world data scenario in which causal inference methods were useful and proved their value as valuable tools in order to better comprehend interpret reality. 


The case chosen was the classical Lalonde, specifically the data from a [posterior paper](https://users.nber.org/~rdehejia/papers/matching.pdf) about the same subject, whose idea was exactly to show how causal inference methods were able to reach similar results like the ones observed during a randomized experimental design by using observational data combined to the treated subjects (i.e successfully emulating a target trial).


The study will be divided into 3 main sections:
- Project Setup: data and dependencies loading, along with the main parameters setting
- Data Analysis: a brief data presentation to give a better sense of how the data look like
- Causal Study: the causal inference implementation on both experimental and observational data


As a footnote, the variables names explanation and the data origin can be found [here](http://users.nber.org/~rdehejia/nswdata2.html).

## Project Setup

In [2]:
import sys
import numpy as np
import pandas as pd
sys.path.insert(1, '../src')
import plotly_utils as pu
import stats_utils as su
import dowhy_utils as du

In [3]:
rct_data = pd.read_stata('../data/raw/nsw_dw.dta').assign(treat = lambda x:x.treat.astype(bool))
observational_data = (
    pd.concat(
        [ 
            pd.read_stata('../data/raw/cps_controls.dta'),
            pd.read_stata('../data/raw/psid_controls.dta'),
        ],
        ignore_index=True
    )
    .assign(treat = lambda x:x.treat.astype(bool))
)
# overview of the data format
rct_data.sample(10)

,data_id,treat,age,education,black,hispanic,married,nodegree,re74,re75,re78
290,Dehejia-Wahba Sample,False,33.0,11.0,1.0,0.0,0.0,1.0,0.000000,0.000000,0.000000
71,Dehejia-Wahba Sample,True,46.0,13.0,1.0,0.0,0.0,0.0,0.000000,0.000000,647.204590
2,Dehejia-Wahba Sample,True,30.0,12.0,1.0,0.0,0.0,0.0,0.000000,0.000000,24909.449219
232,Dehejia-Wahba Sample,False,38.0,10.0,1.0,0.0,0.0,1.0,0.000000,0.000000,12429.910156
395,Dehejia-Wahba Sample,False,21.0,9.0,0.0,0.0,0.0,1.0,2988.412109,1577.166992,1740.198975
313,Dehejia-Wahba Sample,False,33.0,9.0,0.0,1.0,1.0,1.0,0.000000,0.000000,0.000000
154,Dehejia-Wahba Sample,True,42.0,9.0,1.0,0.0,1.0,1.0,0.000000,3058.531006,1294.409058
183,Dehejia-Wahba Sample,True,35.0,8.0,1.0,0.0,1.0,1.0,13732.070312,17976.150391,3786.627930
1,Dehejia-Wahba Sample,True,22.0,9.0,0.0,1.0,0.0,1.0,0.000000,0.000000,3595.894043
127,Dehejia-Wahba Sample,True,23.0,10.0,1.0,0.0,1.0,1.0,0.000000,936.438599,11233.259766


In [4]:
treatment_assign = 'treat'
population_assign = 'data_id'
outcome_variable = 're78'
lalonde_categorical_variables = ['black', 'hispanic', 'married', 'nodegree']
lalonde_continuous_variables = ['age', 'education', 're74', 're75']
cofounders_list = lalonde_categorical_variables + lalonde_continuous_variables

## Data Analysis

The exploratory analysis will consist of the 2 data types available: continuous and categorical (more specifically, dummy variables). Initially, the focus will be on the experimental data; later, the exposure will be finished by demonstrating how the observational data compare to the experimental one.


The main conclusions found here are, as expected, the fundamental role of the randomization on causal inference (and how it can be attested in practice) by ensuring almost no distinction between the treat and control groups; along with how the observational data is more massive and heterogeneous in comparison to the ones found on the experiment.

### Experiment Data

In [7]:
(
    rct_data
    .pipe(pu.group_share_per_category_looper, treatment_assign, category_col_list=lalonde_categorical_variables)
    .pipe(
        pu.prop_stacked_chart, treatment_assign, 'value_share_on_group',
        'category_value', 'category_name'
    )
)

In [8]:
(
    rct_data
    .pipe(pu.continuous_variables_to_boxplot_format, treatment_assign, continuous_col_list=lalonde_continuous_variables)
    .pipe(pu.adjusted_boxplot_per_groups, treatment_assign)
)

### Observational data (per populational sample)

In [9]:
(
    pd.concat([rct_data,observational_data])
    .pipe(pu.group_share_per_category_looper, population_assign, category_col_list=lalonde_categorical_variables)
    .pipe(
        pu.prop_stacked_chart, population_assign, 'value_share_on_group',
        'category_value', 'category_name'
    )
)

In [10]:
(
    pd.concat([rct_data,observational_data])
    .pipe(pu.continuous_variables_to_boxplot_format, population_assign, continuous_col_list=lalonde_continuous_variables)
    .pipe(pu.adjusted_boxplot_per_groups, population_assign)
)

## Causal Study

This section will be focused on Treatment Effect estimations. Initially, a classical estimator (i.e. Independent Samples T-Test) is used to generate an unbiased value for it. On the sequence, the classical example conducted by Lalonde, to demonstrate how using classical OLS regressions on observational data might be inefficient to achieve reasonable estimates (however in a simpler form), is executed for similar purposes. Lastly, some of the simplest Causal Estimators available at Do Why are used aiming to reach a similar conclusion as to the paper's one: it's possible to execute satisfactory causal inference using observational data by applying robust methodology and application.

In [5]:
# rct analysis samples
rct_data_treated = rct_data[rct_data[treatment_assign] == True]
rct_data_control = rct_data[rct_data[treatment_assign] == False]
# fake obs studies samples
fake_obs_study_cps = pd.concat([rct_data_treated, observational_data[observational_data[population_assign] == 'CPS1']])
fake_obs_study_psid = pd.concat([rct_data_treated, observational_data[observational_data[population_assign] == 'PSID']])
fake_obs_studies = [fake_obs_study_cps, fake_obs_study_psid]
# do why causal estimation methods
dowhy_methods = ["backdoor.propensity_score_matching", "backdoor.propensity_score_stratification"]

### Experimental Treatment Effect Estimation

In [8]:
pvalue, power, abs_mean_diff = su.mean_ttest_analyzer(
    rct_data_control[outcome_variable], rct_data_treated[outcome_variable]
)
print(f'The Experimental p-value is {pvalue}')
print(f'The Power of this hypothesis test is {power}')
print(f'The Unbiased Average Treatment Efect is {abs_mean_diff}')

The Experimental p-value is 0.0047875081765774916
The Power of this hypothesis test is 0.4944024937812409
The Unbiased Average Treatment Efect is 1794.34326171875


### Observational Treatment Effect Estimation

In [6]:
ols_causal_est = []
for sample in fake_obs_studies:
    ols_coeffs = su.dataframe_ols_coeffs(sample, outcome_variable, [population_assign], print_stats=True)
    ols_causal_est.append(ols_coeffs[treatment_assign])

                            OLS Regression Results                            
Dep. Variable:                   re78   R-squared:                       0.476
Model:                            OLS   Adj. R-squared:                  0.476
Method:                 Least Squares   F-statistic:                     1631.
Date:                Tue, 05 Jan 2021   Prob (F-statistic):               0.00
Time:                        00:13:06   Log-Likelihood:            -1.6618e+05
No. Observations:               16177   AIC:                         3.324e+05
Df Residuals:                   16167   BIC:                         3.325e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5735.7312    445.176     12.884      0.0

In [33]:
dowhy_causal_est = []
for sample in fake_obs_studies:
    for method in dowhy_methods:
        dowhy_coef = du.dowhy_quick_backdoor_estimator(
            dataframe=sample, outcome=outcome_variable, treatment=treatment_assign,
            cofounders_list=cofounders_list, method_name=method,
            populaton_of_interest='att', view_model=False
        )
        dowhy_causal_est.append(dowhy_coef)

INFO:dowhy.causal_graph:If this is observed data (not from a randomized experiment), there might always be missing confounders. Adding a node named "Unobserved Confounders" to reflect this.
INFO:dowhy.causal_model:Model to find the causal effect of treatment ['treat'] on outcome ['re78']
INFO:dowhy.causal_identifier:Common causes of treatment and outcome:['re75', 'U', 'nodegree', 'black', 'hispanic', 'education', 're74', 'married', 'age']
INFO:dowhy.causal_identifier:Continuing by ignoring these unobserved confounders because proceed_when_unidentifiable flag is True.
INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:[]
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: re78~treat+re75+nodegree+black+hispanic+education+re74+married+age
INFO:dowhy.causal_graph:If this is observed data (not from a randomized experiment), there might always be missing confounders. Adding a node named "Unobserved Confounders

In [35]:
print(f'The Average Do Why Methods Performance was {np.mean(dowhy_causal_est)}')
print(f'The Best Do Why Methods Performance was {np.max(dowhy_causal_est)}')

The Average Do Why Methods Performance was 1029.0057854406855
The Best Do Why Methods Performance was 1773.3112576046506
